Dataset: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis

In [2]:
!pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Import Libraries

import numpy as np
import pandas as pd
from tqdm import tqdm
import zipfile
from zipfile import ZipFile
from collections import Counter
import re



import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


import torch
import torch.nn as nn
import torch.optim as optim
#from torchtext.vocab import Vocab
from torch.utils.data import Dataset, DataLoader
#from torchtext.data.utils import get_tokenizer
#from torchtext.vocab import build_vocab_from_iterator

from transformers import BertTokenizer

text_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

[nltk_data] Downloading package stopwords to C:\Users\Sucheta
[nltk_data]     Jhunjhunwala\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Sucheta
[nltk_data]     Jhunjhunwala\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\Sucheta Jhunjhunwala\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import nltk
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
#from torchtext.vocab import build_vocab_from_iterator
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Sucheta
[nltk_data]     Jhunjhunwala\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sucheta
[nltk_data]     Jhunjhunwala\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Set the path of the ZIP archive and the destination folder where the extracted files will be saved
zip_file_path = 'archive.zip'
destination_folder = '/content/drive/MyDrive/AML'

# Open the ZIP archive using the ZipFile function and extract all its contents to the destination folder using the extractall method
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

In [8]:
train_df=pd.read_csv("twitter_training.csv",names=['Tweet_ID','Entity','Sentiment','Tweet'])
train_df.head()

,Tweet_ID,Entity,Sentiment,Tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [9]:
val_df=pd.read_csv("twitter_validation.csv",names=['Tweet_ID','Entity','Sentiment','Tweet'])
val_df.head()

,Tweet_ID,Entity,Sentiment,Tweet
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [10]:
# Preprocess the text data
def preprocess_text(text):
    text = str(text).lower()  # Convert the text to a string
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\W+', ' ', text)

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

In [11]:
train_df['Tweet'] = train_df['Tweet'].apply(preprocess_text)
val_df['Tweet'] = val_df['Tweet'].apply(preprocess_text)

In [12]:
sentiment_mapping = {'Positive': 0, 'Negative': 1, 'Neutral': 2, 'Irrelevant': 3}

In [13]:
class TextDataset(Dataset):
    def __init__(self, tweets, entities, sentiments, tokenizer, max_len):
        self.tweets = tweets
        self.entities = entities
        self.sentiments = [sentiment_mapping[sentiment] for sentiment in sentiments]  # Update this line
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, item):
        tweet = str(self.tweets[item])
        entity = str(self.entities[item])
        sentiment = self.sentiments[item]

        encoding = self.tokenizer.encode_plus(
            tweet,
            entity,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
            return_overflowing_tokens=True,  # Add this line
        )

        # Handle overflowing tokens
        if 'overflowing_tokens' in encoding:
            encoding['input_ids'] = encoding['input_ids'][:, :self.max_len]
            encoding['attention_mask'] = encoding['attention_mask'][:, :self.max_len]

        return {
            'tweet': tweet,
            'entity': entity,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'sentiments': torch.tensor(sentiment, dtype=torch.long),
        }

In [14]:
class TextDataset(Dataset):
    def __init__(self, tweets, entities, sentiments, tokenizer, max_len):
        self.tweets = tweets
        self.entities = entities
        self.sentiments = [sentiment_mapping[sentiment] for sentiment in sentiments]
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, item):
        tweet = str(self.tweets[item])
        entity = str(self.entities[item])
        sentiment = self.sentiments[item]

        encoding = self.tokenizer.encode_plus(
            tweet,
            entity,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation='only_first',  # Update this line
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'tweet': tweet,
            'entity': entity,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'sentiments': torch.tensor(sentiment, dtype=torch.long),
        }

In [15]:
# Create LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, n_classes, dropout=0.3):
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, n_classes)

    def forward(self, input_ids, attention_mask):
        embedded = self.dropout(self.embedding(input_ids))
        packed_output, (hidden, cell) = self.lstm(embedded)
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        output = self.fc(hidden)

        return output

In [16]:
# Training and validation
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    for d in tqdm(data_loader):
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        sentiments = d['sentiments'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, sentiments)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for d in tqdm(data_loader):
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            sentiments = d['sentiments'].to(device)

            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds.cpu())
            true_labels.extend(sentiments.cpu())

    return accuracy_score(true_labels, predictions)

In [17]:
# Hyperparameters
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-3
MAX_LEN = 256
HIDDEN_DIM = 128
EMBED_DIM = 128
DROPOUT = 0.3
N_CLASSES = 4

In [18]:
# Create DataLoaders
train_dataset = TextDataset(train_df['Tweet'], train_df['Entity'], train_df['Sentiment'], text_tokenizer, MAX_LEN)
val_dataset = TextDataset(val_df['Tweet'], val_df['Entity'], val_df['Sentiment'], text_tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [19]:
# Initialize the model, optimizer, and device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(len(text_tokenizer), EMBED_DIM, HIDDEN_DIM, N_CLASSES, DROPOUT).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [20]:
device

device(type='cpu')

In [21]:
# Train and validate the model
for epoch in range(EPOCHS):
    print(f"Epoch: {epoch+1}")
    train_epoch(model, train_loader, optimizer, device)
    accuracy = evaluate(model, val_loader, device)
    print(f"Validation accuracy: {accuracy:.4f}")

Epoch: 1


100%|██████████| 32/32 [00:05<00:00,  5.53it/s]


Validation accuracy: 0.6980
Epoch: 2


100%|██████████| 32/32 [00:02<00:00, 12.43it/s]


Validation accuracy: 0.8540
Epoch: 3


...

Validation accuracy: 0.9640
Epoch: 9


100%|██████████| 32/32 [00:02<00:00, 11.15it/s]


Validation accuracy: 0.9640
Epoch: 10


100%|██████████| 32/32 [00:02<00:00, 12.05it/s]

Validation accuracy: 0.9690


In [23]:
# Save the model
model_path = "sentiment_LSTM_model.pth"
torch.save(model.state_dict(), model_path)